In [1]:
%pip install PyOpenGL PyOpenGL_accelerate pygame

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys

import numpy as np
import math

# python3 -m pip install

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

# python3 -m pip install pygame
import pygame
from pygame.locals import *

from draw_object import ConfigureEnv

pygame 2.0.1 (SDL 2.0.14, Python 3.7.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2ab)

In [2]:
from scipy import linalg
import numpy as np

sigma = 1
c = sigma
Q = np.array([[c, 0 ,0, 0],[0,0,0,0],[0,0,1,0],[0,0,0,0]])
P = Q

Mp = 1
Mc = 1
l = 1
c_1 = (Mp + Mc)**-1
c_2 = l * Mp * c_1
B = np.array([[0], [c_2 + (c_2*c_1)/(4/3*l-c_2)], [0], [(-c_1)/(4/3*l-c_2)]])

g = 9.81
A = np.array([[0, 1, 0, 0],[0, 0, (-c_2*g)/(4/3*l-c_2), 0],[0, 0, 0, 1],[0, 0, (g)/(4/3*l-c_2), 0]])

R = 1

def calc_K_anal():
    P = linalg.solve_continuous_are(A,B,Q,R)
    K_anal = -B.T @ P
    return K_anal[0]

def calc_K_iter():
    e = 1/1000
    P = Q
    for i in range(0, 10000):
        P = P + e * (A.T @ P + P.T @ A - P @ B @ B.T @ P + Q)
    K_iter = -B.T @ P
    return K_iter[0]

In [3]:
class CartPoleState(object):
    def __init__(self):   
        self.x=0.
        self.x1=0.
        self.th=.2;
        self.th1=0. 

        # init constants
        self.tau = 1/60.;
        Mp = 1
        Mc = 1
        self.l = 1
        self.c1 = 1/(Mp+Mc)
        self.c2 = self.l*Mp/(Mp+Mc)
        self.g = 9.8
        
        # change the dynamicsNoise here
        self.dynamicsNoise = 0.01
    
    def step(self, u):
        the2 = self.g*math.sin(self.th) + math.cos(self.th)*(-self.c1*u-self.c2*self.th1*self.th1*math.sin(self.th));
        the2 /= self.l*4/3 - self.c2*math.cos(self.th)*math.cos(self.th);
        x2 = self.c1*u + self.c2*(self.th1*self.th1*math.sin(self.th) - the2*math.cos(self.th));
        
        self.x   += self.tau*self.x1;
        self.x1  += self.tau*x2;
        self.th  += self.tau*self.th1;
        self.th1 += self.tau*the2;
        
        if(self.dynamicsNoise):
            self.x1 += self.dynamicsNoise*np.random.normal(0,1);
            self.th1 += self.dynamicsNoise*np.random.normal(0,1);


In [4]:
def draw_objects(env, x, theta):
    # draw guide lines
    env.draw_line()  

    # cart
    glColor3f(1,1,1); 
    glPushMatrix()
    glTranslatef(x, 0., 0.)
    env.draw_cube(1., 0.2, 0.2)  
    env.draw_line2()
    
    # pole
    glColor3f(1,0,0); 
    glPushMatrix()
    glRotatef(theta*180./math.pi, 0., 1., 0.)
    glTranslatef(0, 0., 0.5)
    env.draw_cube(0.1, 0.1, 1.)  
    
    glPopMatrix()
    glPopMatrix()
    

In [5]:
def testMove(s):
    # implement the controller gains here
    K = calc_K_anal()
    u = K[0]*s.x + K[1]*s.x1 + K[2]*s.th + K[3]*s.th1;
    s.step(u)
    
    # return translation x and rotation theta
    return s.x, s.th

In [ ]:
def main():
    pygame.init()
 
    display = (1000,750)
    pygame.display.set_mode(display, DOUBLEBUF|OPENGL, RESIZABLE)

    glRotatef(-90.,1.,0, 0)
    glScaled(0.8, 0.8, 0.8);

    env = ConfigureEnv()
    s = CartPoleState()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            env.mouseMove(event);

        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
        
        # calculate and update opengl
        x, th = testMove(s)
        draw_objects(env, x, th)
        
        pygame.display.flip()
        pygame.time.wait(10)

main()

# 2ab)